# Capstone Project Data Science (mdavap)
# Import Library

In [1]:
import sqlalchemy as db 
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

True

# Connect to database

In [2]:
DB_URL = os.getenv('DB_URL')
engine = db.create_engine(DB_URL, echo=True)

# Import our dataset

In [3]:
df = pd.read_csv("../dataset/anime.csv")

In [4]:
df.columns

Index(['id', 'title', 'title_english', 'type', 'source', 'episodes', 'status',
       'airing', 'rating', 'score', 'rank', 'scored_by', 'popularity',
       'favorites', 'members', 'synopsis', 'season', 'year', 'start', 'ending',
       'studios', 'genres'],
      dtype='object')

# Create tables
- anime
    - id (primary key) (integer)
    - mal_id (integer)
    - title (string)
    - title_english (string)
    - type (string)
    - source (string)
    - episodes (integer)
    - status (string)
    - airing (boolean)
    - rating (string)
    - score (float)
    - rank (integer)
    - scored_by (integer)
    - popularity (integer)
    - favorites (integer)
    - members (integer)
    - season (string)
    - year (integer)
    - start (string) (datetime)
    - ending (string) (datetime)
    - synopsis (string)

In [5]:
metadata_obj = db.MetaData() 

anime_table = db.Table( 
    'anime',
    metadata_obj,
    db.Column('id', db.Integer, primary_key=True),   
    db.Column('mal_id', db.Integer),
    db.Column('title', db.Text),
    db.Column('title_english', db.Text),
    db.Column('type', db.Text),
    db.Column('source', db.Text),
    db.Column('episodes', db.Integer),
    db.Column('status', db.Text),
    db.Column('airing', db.Boolean),
    db.Column('rating', db.Text),
    db.Column('score', db.Float),
    db.Column('rank', db.Integer),
    db.Column('scored_by', db.Integer),
    db.Column('popularity', db.Integer),
    db.Column('favorites', db.Integer),
    db.Column('members', db.Integer),
    db.Column('seasons', db.Text),
    db.Column('year', db.Integer),
    db.Column('start', db.DateTime),
    db.Column('ending', db.DateTime),
    db.Column('synopsis', db.Text)
) 

- genre_name
    - id (primary key) (integer)
    - name (string)

- studio_name
    - id (primary key) (integer)
    - name (string)

In [6]:
genre_name_table = db.Table( 
    'genre_name',
    metadata_obj,
    db.Column('id', db.Integer, primary_key=True),   
    db.Column('name', db.Text),
)

studio_name_table = db.Table( 
    'studio_name',
    metadata_obj,
    db.Column('id', db.Integer, primary_key=True),   
    db.Column('name', db.Text),
)

- genres
    - id (primary key) (integer)
    - anime_id (foreign key -> anime.id) (integer)
    - genre_id (foreign key -> genre_name.id) (integer)

- studios
    - id (primary key) (integer)
    - anime_id (foreign key -> anime.id) (integer)
    - studio_id (foreign key -> studio_name.id) (integer)

In [7]:
genres_table = db.Table( 
    'genres',
    metadata_obj,
    db.Column('id', db.Integer, primary_key=True),  
    db.Column('anime_id', db.Integer, db.ForeignKey('anime.id')),   
    db.Column('genre_id', db.Integer, db.ForeignKey('genre_name.id')),   
)

studios_table = db.Table( 
    'studios',
    metadata_obj,
    db.Column('id', db.Integer, primary_key=True),  
    db.Column('anime_id', db.Integer, db.ForeignKey('anime.id')),   
    db.Column('studio_id', db.Integer, db.ForeignKey('studio_name.id')),   
)

In [8]:
metadata_obj.create_all(engine)

2025-01-18 14:59:20,070 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-01-18 14:59:20,070 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-18 14:59:20,131 INFO sqlalchemy.engine.Engine select current_schema()
2025-01-18 14:59:20,132 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-18 14:59:20,190 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-01-18 14:59:20,191 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-18 14:59:20,248 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-18 14:59:20,252 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname